In [ ]:
import time
from utils import reload_module

reload_module("epsql")
import epsql

engine = epsql.Engine()

In [ ]:
def add_highest_overlap_crosswalk(self, dest_table_name, dest_col, src_table_name, src_col):
    # Assumes dest_col should be text
    before = time.time()
    print(f'Adding {dest_table_name}.{dest_col} as crosswalk to {src_table_name}.{src_col} selecting highest overlap...')
    self.execute(f'alter table {dest_table_name} add if not exists {dest_col} text;')
    cmd = f"""
    update {dest_table_name} as dest
        set {dest_col} = (
            select src.{src_col}
            from {src_table_name} as src
            where st_intersects(src.geom, dest.geom) and not st_touches(src.geom, dest.geom)
            order by st_area(st_intersection(src.geom, dest.geom)) desc
            limit 1)
        --where dest.statefp10='42' and dest.countyfp10='003'
        where dest.statefp10='31'
    """
    rowcount = self.execute_update(cmd)
    duration = time.time() - before
    print(f'Completed {rowcount} rows in {duration:.1f} seconds ({rowcount/duration:.1f} rows/sec)')

blarf = add_highest_overlap_crosswalk(engine,
    dest_table_name="tiger_wgs84.tl_2010_tabblock10",
    dest_col="tract_2019",
    src_table_name="tiger_wgs84.tl_2019_tract",
    src_col="geoid")

#engine.execute_returning_df("select * from tiger_wgs84.tl_2019_tract limit 5")

In [ ]:
table_name = "tiger_wgs84.tl_2010_tabblock10"
table_sans_schema = epsql.get_table_name(table_name)
z=engine.execute(f"CREATE INDEX IF NOT EXISTS {table_sans_schema}_statefp10_idx ON {table_name} (statefp10)", verbose=True)
z=engine.execute(f"CREATE INDEX IF NOT EXISTS {table_sans_schema}_statefp10_countyfp10_idx ON {table_name} (statefp10, countyfp10)", verbose=True)


In [ ]:
engine.execute_count("SELECT COUNT(*) FROM tiger_wgs84.tl_2010_tabblock10")

In [ ]:
engine.execute_returning_df("SELECT * FROM tiger_wgs84.tl_2010_tabblock10 limit 10")

In [ ]:
engine.execute_count("SELECT count(*) FROM tiger_wgs84.tl_2010_tabblock10 where left(geoid10,11) <> tract_2019")

In [ ]:
gdf = engine.execute_returning_gdf("SELECT * FROM tiger_wgs84.tl_2010_tabblock10 where statefp10='42' and countyfp10='003' and left(geoid10,11) <> tract_2019")

In [ ]:
#import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [10, 5]
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 15]
gdf.plot()
